In [1]:
import torch
from gensim.models import KeyedVectors

from data_iterator import DataIterator
from utils import load_pickled_obj

In [2]:
wv = KeyedVectors.load("embedding/amazon_sample.wv")
vocab_size = len(wv.vocab)
embedding_dim = wv.vector_size
embedding_matrix = wv.vectors

In [7]:
class WordEncoder(torch.nn.Module):
    def __init__(self, embedding_matrix, hidden_size=50):
        super(WordEncoder, self).__init__()
        embedding_dim = embedding_matrix.shape[1]
        self.embedding = torch.nn.Embedding.from_pretrained(
            embeddings=torch.FloatTensor(embedding_matrix), freeze=True,
        )
        self.gru = torch.nn.GRU(
            input_size=embedding_dim, hidden_size=hidden_size, bidirectional=True
        )
        self.word_context_vector = torch.nn.Parameter(
            torch.Tensor(2 * hidden_size, 2 * hidden_size)
        )

    def forward(self, input, hidden_state):
        output = self.embedding(input)
        return self.gru(output.float(), hidden_state)

In [4]:
dataset = load_pickled_obj("data/amazon_train_sample.pkl")

In [17]:
encoder = WordEncoder(embedding_matrix)
it = iter(DataIterator(dataset, wv.vocab))
labels, batch = next(it)
batch = torch.LongTensor(batch)
batch = batch.permute(1, 0, 2)
batch.shape
i = batch[0]
i = i.permute(1, 0)
output, h_output = encoder(i, torch.zeros(2, 64, 50))
print(output.shape)
print(h_output.shape)










  0%|          | 0/328500 [00:00<?, ?it/s]

torch.Size([184, 64, 100])
torch.Size([2, 64, 50])
